In [1]:
import subprocess
import os

netlist_template = """
*CONDUCTION NMOS W1

.INCLUDE ../../model/32nm_HP.pm
.OPTIONS GMIN=1e-020 ABSTOL=1e-18

*Definizione dei parametri
.PARAM Lmin=22n
.PARAM Wmin=22n
.PARAM Ldiff=44n

.TEMP 85

.PARAM N={n}

*Descrizione della cella
Mn drain gate source body nmos W={{Wmin*N}} L={{Lmin}} AS={{Wmin*N*Ldiff}} AD={{Wmin*N*Ldiff}} PS={{2*(Ldiff+Wmin*N)}} PD={{2*(Ldiff+Wmin*N)}}
Vd 	drain	0		0
Vg 	gate	alim		0
Vs 	source	0		0
Vb 	body	0		0
Vdd 	alim	0		0

*Definizione del tipo di analisi
.CONTROL
let voltage=-0.05
let Vddbasic=1.0

while voltage le Vddbasic
  let voltage = voltage + 0.05
  alter Vdd = voltage
  dc TEMP 85 86 10
  print abs(V(alim)) V(drain) V(gate) V(source) V(body) I(Vd) I(Vg) I(Vs) I(Vb)

end

.ENDC
.END

"""

n_values = [1, 2, 3, 4, 6, 8]

output_dir = "netlists_varying_width"

# make the directory if it doesn't exist
os.makedirs(output_dir, exist_ok=True)

outs = []
for n in n_values:
    netlist_content = netlist_template.format(n=n)
    
    filename = f"{output_dir}/nmos_off_{n}.net"
    
    with open(filename, 'w') as file:
        file.write(netlist_content)
    
    p = subprocess.run(["ngspice", filename] , encoding='utf-8', 
stdout=subprocess.PIPE)
    windstr = p.stdout
    outs.append(windstr)

Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run
Note: No ".plot", ".print", or ".fourier" lines; no simulations run


In [2]:
lines = windstr.split('\n')
# make a matrix out of every 10-18 lines
matrix = []
for o in outs:
    lines = o.split('\n')
    curr = []
    # make a matrix out of every 10-18 lines
    for i in range(10, len(lines), 13):
        curr.append(lines[i:i+9])
    matrix.append(curr)

In [3]:
# make directory 
dir_name = "results"
os.makedirs(dir_name, exist_ok=True)

filename = f"{dir_name}/leakage_width_"
for i, n in enumerate(n_values):
    with open(filename + str(n) + ".csv", 'w') as file:
        file.write("Valim,Vd,Vg,Vs,Vb,Id,Ig,Is,Ib\n") 
        for j in range(len(matrix[i])):
            for k in range(9):
                value = matrix[i][j][k].split(' ')[-1]
                file.write(value + ",")
                
            file.write("\n")